# Text generator based on RNN

### Import libraries

In [1]:
import tensorflow as tf
import numpy as np
import random

### Configurations & data encoding

In [2]:
vocab = (" $%'()+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ"
            "\\^_abcdefghijklmnopqrstuvwxyz{|}\n")
TestText="""
Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal.

Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battle-field of that war. We have come to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this.

But, in a larger sense, we can not dedicate—we can not consecrate—we can not hallow—this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here to the unfinished work which they who fought here have thus far so nobly advanced. It is rather for us to be here dedicated to the great task remaining before us—that from these honored dead we take increased devotion to that cause for which they gave the last full measure of devotion—that we here highly resolve that these dead shall not have died in vain—that this nation, under God, shall have a new birth of freedom—and that government of the people, by the people, for the people, shall not perish from the earth.
"""
graphPath=r".\graphs"
num_step=len(TestText)-1
batchSize=1
modelParamPath=r".\model_checkpoints"
def Encode(string):
    tensor=[vocab.find(ch)+1 for ch in string]
    tensor=tf.one_hot(tensor,depth=len(vocab)+1,on_value=1.0,off_value=0.0,axis=-1, dtype=tf.float32)
    with tf.Session() as sess:
        nparray=tensor.eval()
    return nparray
def Decode(nparray):
    return "".join([vocab[index-1] if index>0 else "[INVALID]" for index in np.argmax(nparray,axis=1)])
X=Encode(TestText)

Y=X[1:]
X=X[:-1]

### Define model class

In [3]:
class MyRNN():
    def __init__(self,inputVectorDim,modelCheckpointPath,stateSize=100,scope="RNN",outputVectorDim=None):
        if outputVectorDim==None:
            outputVectorDim=inputVectorDim
        self.__inputVectorDim=inputVectorDim
        self.__outputVetorDim=outputVectorDim
        self.__stateSize=stateSize
        self.__mdlCkptPath=modelCheckpointPath
        with tf.variable_scope(scope) as modelScope:
            with tf.variable_scope("structure") as structureScope:
                #One batch at a time
                self.inputs=tf.placeholder(dtype=tf.float32,shape=[None,inputVectorDim])
                self.outputs=tf.placeholder(dtype=tf.float32,shape=[None,outputVectorDim])
                self.__cell=tf.contrib.rnn.GRUCell(num_units=stateSize)
                self.__initState = tf.placeholder_with_default(self.__cell.zero_state(batch_size=1,dtype=tf.float32),
                                                        shape=[None,self.__stateSize])
                #S20:State to output
                self.__wS2O=tf.get_variable(name="Weight_s2o",shape=[stateSize,outputVectorDim],
                                            initializer=tf.truncated_normal_initializer())
                self.__bS2O=tf.get_variable(name="Bias_s2o",shape=[1,outputVectorDim],
                                            initializer=tf.truncated_normal_initializer())
                #add a "batch dimension" to the front
                cellInputs=tf.reshape(tensor=self.inputs,shape=[1,-1,inputVectorDim])
                self.__stateOutput,self.finalState=tf.nn.dynamic_rnn(cell=self.__cell,inputs=cellInputs,
                                                     dtype=tf.float32,initial_state=self.__initState)
                outputs=tf.reshape(tensor=self.__stateOutput, shape=[-1, stateSize])#Remove the batch dimension ((
                self.netOutputs=tf.add(tf.matmul(outputs,self.__wS2O),self.__bS2O)
            with tf.name_scope("training"):
                loss=tf.losses.softmax_cross_entropy(logits=self.netOutputs,onehot_labels=self.outputs)
                loss=tf.reduce_mean(loss,name="loss")
                self.optimizer=tf.train.AdamOptimizer(learning_rate=0.001,name="optimizer").minimize(loss)
            with tf.name_scope("summary") as sumScope:
                lossSum=tf.summary.scalar(tensor=loss,name="loss")
                self.summary_op=tf.summary.merge(tf.get_collection(tf.GraphKeys.SUMMARIES,sumScope))
    
    def onlineInference(self,timeSteps=100,state=None,seed=None,modelParamPath=None):
        feedDict={}
        if modelParamPath==None:
            modelParamPath=self.__mdlCkptPath
        if state!=None:
            feedDict[self.__initState]=state
        if seed==None:
            #Create a random seed
            seed=tf.one_hot([random.randint(0,self.__inputVectorDim)],
                            depth=self.__inputVectorDim,on_value=1.0,off_value=0.0,axis=-1,dtype=tf.float32)
            with tf.Session() as sess:
                seed=sess.run(seed)
        feedDict[self.inputs]=seed
        seq=[]
        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state(self.__mdlCkptPath)
            print("Loading model")
            if ckpt and ckpt.model_checkpoint_path:  
                saver.restore(sess,ckpt.model_checkpoint_path)
                print("Successfully loaded")
            for length in range(timeSteps):
                input_,state=sess.run(fetches=[self.netOutputs,self.finalState],
                                      feed_dict=feedDict)
                #Since the input vectors are discrete, so we need to convert it into a
                #new one hot vector
                input_=tf.one_hot(indices=[int(np.argmax(input_[0]))],depth=self.__outputVetorDim,
                                  on_value=1.0,off_value=0.0,axis=-1,dtype=tf.float32).eval()
                seq.append(input_)
                feedDict[self.inputs],feedDict[self.__initState]=input_,state
            seq=tf.concat(values=seq,axis=0).eval()
        return seq,state

### Make an instance

In [4]:
Test=MyRNN(scope="RNN",inputVectorDim=len(vocab)+1,modelCheckpointPath=modelParamPath)

### Training

In [5]:
n_epochs=1000
with tf.Session() as sess:
    writer=tf.summary.FileWriter(logdir=graphPath,graph=sess.graph)
    saver=tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    for i in range(n_epochs):
        _,summary=sess.run([Test.optimizer,Test.summary_op],feed_dict={Test.inputs:X,Test.outputs:Y})
        writer.add_summary(summary,global_step=i+1)
        if i%100==0:
            saver.save(sess=sess,save_path=modelParamPath+r"\rnnMDL",global_step=i+1)
            result=sess.run(Test.netOutputs,{Test.inputs:X})
            print("Epoch: {}".format(i+1))
            print(Decode(result))
    result=sess.run(Test.netOutputs,{Test.inputs:X})
    print("Final result after {} epoch(s)".format(i+1))
    print(Decode(result))
    writer.close()
    saver.save(sess=sess,save_path=modelParamPath+r"\rnnMDL",global_step=n_epochs)

Epoch: 1
??U?U???UUUU??????UUU??UUUU???U?U?U?L????UU?UUUUULL?LLLLLLLL??L????????????????????????????????U?UUU??????UUUUUU????????????U7??U???(??UU?UU?????????????Z??????UUUUUUUUUUUUUU77???????????UUUUUUUUUU7??????UUUU?U???????????UU?????????LUUUULL???????????U???????????????????U??UU????????????????7U?U7???U???U???UUU??UU?UUUUUUUUU????UUUU?UUU?-UUU?UUZ?U???????????????L??UUUUUUUUUU?UU?77?U?????UU???????????????????????????????UU????????????UUU?LLZU???U?((UUUUUUUUU(UUUUU??UU?????????????????????????????????????????UULUUUU????????????????UU?L????????Z?ZZ?????????????????????????UUUUUUUUUUUUUU7?????????????UUUUUU77??????U??UUUU?UUUUUU77???????ZZZZZ???????U??????LL?UUUUUUUUUUUU?????????????7????????U?LLLUU??UUUUUUUUUUUUUUUUUUUUU??????????UUUUUUUUU?UUUU???UUU?U??????U??UUUUU???LLL??UUUZ????????????????UU??UUUUUZZ?UUUUUUUUUUUU???U???????UUUUUUUU?????????UUUUUUUUUUUUL?????(???????(UUUUUU??????U??????(?????????U???UUUUUU?UUUUUU777?U7?(UUUUUUUUL((L?L??????????UU?????????(????L??LLLLL?L(UUU(LUUU

### Test online inference

In [6]:
result,_=Test.onlineInference(timeSteps=len(TestText)-1,seed=X[:1])
print(Decode(X[:1])+Decode(result))

e:\python35\lib\site-packages\ipykernel\__main__.py:42: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


Loading model
Successfully loaded

Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal.

Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battle-field of that war. We have come to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this.

But, in a larger sense, we can not dedicate[INVALID]we can not consecrate[INVALID]we can not hallow[INVALID]this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here to the

### Evaluate transition performance of the model

In [7]:
with tf.Session() as sess:
    print("Loading model")
    ckpt=tf.train.get_checkpoint_state(modelParamPath)
    if ckpt and ckpt.model_checkpoint_path:  
        saver.restore(sess,ckpt.model_checkpoint_path)
        print("Successfully loaded")
    result=sess.run(Test.netOutputs,{Test.inputs:X})
print(Decode(result))

Loading model
Successfully loaded
Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal.

Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battle-field of that war. We have come to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this.

But, in a larger sense, we can not dedicate[INVALID]we can not consecrate[INVALID]we can not hallow[INVALID]this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here to the 